# Visualisation Data Generation
___

This notebook allows to generate data to be displayed on the wep app. First, we generate data based on a night of sleep from our dataset, then we generate data based on a night of sleep acquired with an OpenBCI (William's night).

## Data
___

This notebook aims to generate data that will be used by our visualisations. We generate, for a night recording and its sleep stage scoring, two files:
- `hypnogram.csv`:

    Each row contains the following tuple (timestamp:`int`, sleep stage:`int`) for each 30 seconds epoch of the night. The sleep stage equivalences are: 
    ```
"W": 0,
"N1": 1,
"N2": 2,
"N3": 3,
"REM": 4
    ```


- `spectrograms.json`:

    Dictionnary that contains all information needed to display both spectrograms for each EEG channel. All 30 seconds epochs corresponds to those found in `hypnogram.csv`.
    
    ```
{
    "frequencies": [float],  # 75 frequencies corresponding to the y-axis
    "Fpz-Cz": [              # List for all 30 second epochs
        [float]              # 75 power values (in log scale) associated to the frequencies
    ],
    "Pz-Oz": [               # List for all 30 second epochs
        [float               # 75 power values (in log scale) associated to the frequencies
    ]
}
    ```



In [ ]:
%matplotlib inline

import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from time import altzone
from pytz import timezone
from json import dump
import os

from utils import fetch_data
from constants import (
    SLEEP_STAGES_VALUES,
    DATASET_SLEEP_STAGES_VALUES,
    N_STAGES,
    EEG_CHANNELS,
    EPOCH_DURATION,
    SAMPLING_FREQ,
    ELECTROPHYSIOLOGIST_SLEEP_STAGES,
)

## 1. Dataset 
___

### Selected data
___
**Analyse my data**: Currently visualized subject in that page is subject **55**, night **1**.

**Performance**:
- Currently visualized hypnogram in Classifier vs Physionet and Electrophysiologist vs Physionet is subject **55**, night **1**. 
- Currently visualized hypnogram in Classifier vs Electrophysiologist is subject **55**, night **1**. 

In [ ]:
NB_EPOCHS_AWAKE_MORNING = 60
    
NYQUIST_FREQ = SAMPLING_FREQ/2
MAX_TIME = EPOCH_DURATION - 1. / SAMPLING_FREQ  # tmax in included

SUBJECT = [3]
NIGHT = [2]
FOLDER_NAME = f"./data/subject{SUBJECT[0]}-night{NIGHT[0]}"

### Extract data
___

In [ ]:
file_names = fetch_data(subjects=SUBJECT, recording=NIGHT)
print(file_names)

raw_data = mne.io.read_raw_edf(file_names[0][0], preload=True, stim_channel=None, verbose=False)
annot = mne.read_annotations(file_names[0][1])
raw_data.set_annotations(annot, emit_warning=False)


In [ ]:
df_records = pd.read_csv("data/recordings-info.csv")
df_records.head(2)

In [ ]:
info = df_records[(df_records['subject'] == SUBJECT[0]) & (df_records['night'] == NIGHT[0])]
print(info)

# Number of seconds since file began
closed_lights_time = info['LightsOffSecond'].values[0]
woke_up_time = closed_lights_time + info['NightDuration'].values[0] + NB_EPOCHS_AWAKE_MORNING*EPOCH_DURATION

raw_data.crop(tmin=closed_lights_time, tmax=min(woke_up_time, raw_data.times[-1]))

events, annot_event_id = mne.events_from_annotations(
    raw_data,
    event_id=DATASET_SLEEP_STAGES_VALUES,
    chunk_duration=EPOCH_DURATION,
    verbose=False)

# Few files do not have N3 sleep (i.e. SC4202EC-Hypnogram), so we have to filter out key-value pairs that are not in the annotations.
event_id = { 
    event_key: SLEEP_STAGES_VALUES[event_key] 
    for event_key in SLEEP_STAGES_VALUES
    if SLEEP_STAGES_VALUES[event_key] in annot_event_id.values()
}

In [ ]:
epochs = mne.Epochs(
    raw=raw_data,
    events=events,
    event_id=event_id,
    tmin=0.,
    tmax=MAX_TIME,
    preload=True,
    baseline=None,
    verbose=False)

epochs.drop_channels([
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker'])

y = np.array([event[-1] for event in epochs.events])

In [ ]:
print("Number of epochs with raw data:           ", epochs.get_data().shape)
print("Number of staged epochs:                  ", y.shape)
print("Time between get out of bed and bed time: ", (woke_up_time - closed_lights_time)/EPOCH_DURATION)
print("Night duration with extra morning time:   ", info['NightDuration'].values[0] + NB_EPOCHS_AWAKE_MORNING*EPOCH_DURATION)
print("Time of bedtime:                          ", info['LightsOff'].values[0])

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)

raw_data.copy().pick('EEG Fpz-Cz').plot(butterfly=True, duration=info['NightDuration'].values[0]);

In [ ]:
raw_data.copy().pick('EEG Pz-Oz').plot(butterfly=True, duration=info['NightDuration'].values[0]);

### Generate hypnogram
___

In [ ]:
y_with_offset = np.array([(index*EPOCH_DURATION, stage) for index, stage in enumerate(y)])

plt.rcParams["figure.figsize"] = (20,5)
plt.plot([y[0]/3600 for y in y_with_offset], [y[1] for y in y_with_offset])
plt.xlabel("Time since bed time (hours)")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()

In [ ]:
y_with_offset

In [ ]:
info['LightsOff'].values[0]

In [ ]:
datetime_bedtime = datetime.strptime(info['LightsOff'].values[0], "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone('utc'))
print(datetime_bedtime.timestamp())

y_with_timestamp = np.array([(
    int((datetime_bedtime + timedelta(seconds=index*EPOCH_DURATION)).timestamp()),
    int(stage))
    for index, stage in enumerate(y)])
y_with_timestamp

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plt.plot([y[0] for y in y_with_timestamp], [y[1] for y in y_with_timestamp])
plt.xlabel("Unix timestamp")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()

#### Save hypnogram to file
___

In [ ]:
if not os.path.exists(FOLDER_NAME):
    os.mkdir(FOLDER_NAME);

np.savetxt(
    f"{FOLDER_NAME}/hypnogram.csv",
    y_with_timestamp,
    fmt=['%10.0i', '%1.0i'],
    delimiter=',',
    header="timestamp,sleep_stage",
    comments='')

### Generate spectrogram
___

In [ ]:
epochs.plot_psd(fmin=0.5, fmax=30., dB=True)

In [ ]:
psds, freqs = mne.time_frequency.psd_welch(epochs, fmin=0.5, fmax=30.)
# Convert psds into dB scale
psds = 10 * np.log10(np.maximum(psds, np.finfo(float).tiny))

psds.shape, freqs.shape

In [ ]:
# Plot histogram
plt.rcParams["figure.figsize"] = (20,1.5)
plt.plot([y[0] for y in y_with_offset], [y[1] for y in y_with_offset])
plt.xlabel("Time since bed time (seconds)")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()
plt.show()

# Plot spectrogram of Fpz-Cz
plt.rcParams["figure.figsize"] = (20,5)
plt.pcolormesh(
    [y[0] for y in y_with_offset],
    freqs,
    psds[:,0,:].T)
plt.ylim([0,30])
plt.xlabel('Time (s)'), plt.ylabel('Frequency (Hz)'),  plt.title('Spectrogram for Fpz-Cz')
plt.show()

# Plot spectrogram of Pz-Oz
plt.pcolormesh(
    [y[0] for y in y_with_offset],
    freqs,
    psds[:,1,:].T)
plt.ylim([0,30])
plt.xlabel('Time (s)'), plt.ylabel('Frequency (Hz)'),  plt.title('Spectrogram for Pz-Oz')
plt.show()

#### Save spectrogram to file
___

In [ ]:
spectrograms = {
    'Frequencies': freqs.tolist(),
    'Fpz-Cz': psds[:,0,:].tolist(),
    'Pz-Oz': psds[:,1,:].tolist()
}

with open(f"{FOLDER_NAME}/spectrograms.json", 'w') as json_file:
    dump(spectrograms, json_file)

## 2. Data from OpenBCI 
___

### Selected data
___

The data used will be of William's night.

In [ ]:
FOLDER_NAME = "data/williamharvey-night1"

psg_file_name = f"{FOLDER_NAME}/william-recording.edf"
hypno_file_name = f"{FOLDER_NAME}/william-hypnogram.edf"
hypno_predicted_file_name = f"{FOLDER_NAME}/william-hypnogram-predicted.npy"
journal_file_name = f"{FOLDER_NAME}/william-journal.csv"

openbci_raw_signal = mne.io.read_raw_edf(psg_file_name, preload=True, stim_channel=None, verbose=False)
openbci_hypnogram_labelled = mne.read_annotations(hypno_file_name)
openbci_hypnogram_predicted = np.loadtxt(hypno_predicted_file_name)
openbci_journal_df = pd.read_csv(journal_file_name)

In [ ]:
openbci_journal_df.iloc[:, :] = openbci_journal_df.iloc[:, :].apply(pd.to_datetime)

openbci_journal_df, openbci_journal_df.dtypes

In [ ]:
# Number of seconds since file began
closed_lights_seconds = (openbci_journal_df['bed_time'] - openbci_journal_df['start_time']).dt.seconds
wake_up_seconds = (openbci_journal_df['wake_up_time'] - openbci_journal_df['start_time']).dt.seconds

openbci_raw_signal.resample(SAMPLING_FREQ)
openbci_raw_signal.filter(l_freq=0.5, h_freq=None)

openbci_raw_signal.set_annotations(
    openbci_hypnogram_labelled,
    emit_warning=False
)
openbci_raw_signal.crop(
    tmin=closed_lights_seconds[0],
    tmax=min(wake_up_seconds[0], openbci_raw_signal.times[-1])
)

events, annot_event_id = mne.events_from_annotations(
    openbci_raw_signal,
    event_id=ELECTROPHYSIOLOGIST_SLEEP_STAGES,
    chunk_duration=EPOCH_DURATION,
    verbose=False)

# Few files do not have N3 sleep (i.e. SC4202EC-Hypnogram), so we have to filter out key-value pairs that are not in the annotations.
event_id = { 
    event_key: SLEEP_STAGES_VALUES[event_key] 
    for event_key in SLEEP_STAGES_VALUES
    if SLEEP_STAGES_VALUES[event_key] in annot_event_id.values()
}

epochs = mne.Epochs(
    raw=openbci_raw_signal,
    events=events,
    event_id=event_id,
    tmin=0.,
    tmax=MAX_TIME,
    preload=True,
    baseline=None,
    verbose=False)

epochs.drop_channels([
 'EOG1-EOG2',
 'EMG1-EMG2',
])

y = np.array([event[-1] for event in epochs.events])

epochs.info

In [ ]:
# crop predictions

openbci_hypnogram_predicted = openbci_hypnogram_predicted[
    int(closed_lights_seconds / EPOCH_DURATION):
    int(wake_up_seconds / EPOCH_DURATION)
]

In [ ]:
print("Number of epochs with raw data:           ", epochs.get_data().shape)
print("Number of staged epochs (labelled):       ", y.shape)
print("Number of staged epochs (predicted):      ", openbci_hypnogram_predicted.shape)

print("Time between get out of bed and bed time: ", (openbci_journal_df['wake_up_time'] - openbci_journal_df['bed_time'])[0])
print("Total effective sleep:                    ", timedelta(seconds=y[y != SLEEP_STAGES_VALUES['W']].shape[0] * EPOCH_DURATION))

### Generate hypnogram - Labelled
___

In [ ]:
y_with_timestamp = np.array([(
        int((openbci_journal_df.loc[0,'bed_time'] + timedelta(seconds=index*EPOCH_DURATION)).timestamp()),
        str(list(SLEEP_STAGES_VALUES.keys())[list(SLEEP_STAGES_VALUES.values()).index(stage)]) # gets the label for the stage index
    ) for index, stage in enumerate(y)])
y_with_timestamp

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plt.plot([y[0] for y in y_with_timestamp], [SLEEP_STAGES_VALUES[y[1]] for y in y_with_timestamp])
plt.xlabel("Unix timestamp")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()

#### Save hypnogram to file
___

In [ ]:
if not os.path.exists(FOLDER_NAME):
    os.mkdir(FOLDER_NAME);

np.savetxt(
    f"{FOLDER_NAME}/hypnogram-labelled.csv",
    y_with_timestamp,
    fmt='%s',
    delimiter=',',
    header="timestamp,sleepStage",
    comments='')

### Generate hypnogram - Predicted
___

In [ ]:
y_with_timestamp = np.array([(
        int((openbci_journal_df.loc[0,'bed_time'] + timedelta(seconds=index*EPOCH_DURATION)).timestamp()),
        str(list(SLEEP_STAGES_VALUES.keys())[list(SLEEP_STAGES_VALUES.values()).index(stage)]) # gets the label for the stage index
    ) for index, stage in enumerate(openbci_hypnogram_predicted)])
y_with_timestamp

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plt.plot([y[0] for y in y_with_timestamp], [SLEEP_STAGES_VALUES[y[1]] for y in y_with_timestamp])
plt.xlabel("Unix timestamp")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()

#### Save hypnogram to file
___

In [ ]:
if not os.path.exists(FOLDER_NAME):
    os.mkdir(FOLDER_NAME);

np.savetxt(
    f"{FOLDER_NAME}/hypnogram-predicted.csv",
    y_with_timestamp,
    fmt='%s',
    delimiter=',',
    header="timestamp,sleepStage",
    comments='')

### Generate spectrogram
____

In [ ]:
epochs.plot_psd(fmin=0.5, fmax=30., dB=True)

In [ ]:
psds, freqs = mne.time_frequency.psd_welch(epochs, fmin=0.5, fmax=30.)
# Convert psds into dB scale
psds = 10 * np.log10(np.maximum(psds, np.finfo(float).tiny))

psds.shape, freqs.shape

In [ ]:
# Plot histogram
plt.rcParams["figure.figsize"] = (20,1.5)
plt.plot([y[0] for y in y_with_timestamp], [y[1] for y in y_with_timestamp])
plt.xlabel("Time since bed time (seconds)")
plt.ylabel("Sleep stage")
plt.gca().invert_yaxis()
plt.show()

# Plot spectrogram of Fpz-Cz
plt.rcParams["figure.figsize"] = (20,5)
plt.pcolormesh(
    [y[0] for y in y_with_timestamp],
    freqs,
    psds[:,0,:].T)
plt.ylim([0,30])
plt.xlabel('Time (s)'), plt.ylabel('Frequency (Hz)'),  plt.title('Spectrogram for Fpz-Cz')
plt.show()

# Plot spectrogram of Pz-Oz
plt.pcolormesh(
    [y[0] for y in y_with_timestamp],
    freqs,
    psds[:,1,:].T)
plt.ylim([0,30])
plt.xlabel('Time (s)'), plt.ylabel('Frequency (Hz)'),  plt.title('Spectrogram for Pz-Oz')
plt.show()

#### Save spectrogram to file
___

In [ ]:
spectrograms = {
    'frequencies': freqs.tolist(),
    'fpz-cz': psds[:,0,:].tolist(),
    'pz-oz': psds[:,1,:].tolist()
}

with open(f"{FOLDER_NAME}/spectrograms.json", 'w') as json_file:
    dump(spectrograms, json_file)